### Testing quality of predictions

We now have a function that can predict the price for any living space we want to list as long as we know the number of people it can accommodate. The function we wrote represents a machine learning model, which means that it outputs a prediction based on the input to the model.


A simple way to test the quality of your model is to:

split the dataset into 2 partitions:
1. the training set: contains the majority of the rows (75%)
2. the test set: contains the remaining minority of the rows (25%)

use the rows in the training set to predict the price value for the rows in the test set
add new column named predicted_price to the test set
compare the predicted_price values with the actual price values in the test set to see how accurate the predicted values were.

This validation process, where we use the training set to make predictions and the test set to predict values for, is known as train/test validation. Whenever you're performing machine learning, you want to perform validation of some kind to ensure that your machine learning model can make good predictions on new data. While train/test validation isn't perfect, we'll use it to understand the validation process, to select an error metric, and then we'll dive into a more robust validation process later in this course.

Let's modify the predict_price function to use only the rows in the training set, instead of the full dataset, to find the nearest neighbors, average the price values for those rows, and return the predicted price value. Then, we'll use this function to predict the price for just the rows in the test set. Once we have the predicted price values, we can compare with the true price values

### Instructions
Within the predict_price function, change the Dataframe that temp_df is assigned to.
1. Change it from dc_listings to train_df, so only the training set is used.

2. Use the Series method apply to pass all of the values in the accommodates column from test_df through the predict_price function.

3. Assign the resulting Series object to the predicted_price column in test_df.

In [29]:
import pandas as pd
import numpy as np
dc_listings = pd.read_csv("dc_airbnb.csv")
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')
train_df = dc_listings.iloc[0:2792]
test_df = dc_listings.iloc[2792:]
def predict_price(new_listing):
    ## DataFrame.copy() performs a deep copy
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - new_listing))
    
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)
test_df['predicted_price'] = test_df['accommodates'].apply(lambda x:predict_price(x)) 

<ipython-input-29-9a8732bfb548>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_price'] = test_df['accommodates'].apply(lambda x:predict_price(x))


In [8]:
#test_df['predicted_price'] = test_df['accommodates'].apply(predict_price) 

In [30]:
test_df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,predicted_price
2792,20%,75%,1,2,Entire home/apt,0.0,1.0,1.0,120.0,NaN,NaN,1,1125,8,38.922187,-77.032475,Washington,20009,DC,104.0
2793,100%,25%,2,3,Entire home/apt,2.0,2.0,1.0,140.0,$75.00,$150.00,2,1125,7,38.931681,-77.044739,Washington,20010,DC,177.4
2794,NaN,NaN,1,4,Entire home/apt,2.0,1.0,1.0,299.0,NaN,NaN,2,1125,5,38.933765,-77.031488,Washington,20010,DC,145.8
2795,100%,100%,1,3,Entire home/apt,1.0,1.0,1.0,85.0,$30.00,$250.00,1,92,2,38.925692,-77.032616,Washington,20009,DC,177.4
2796,100%,100%,1,6,Entire home/apt,2.0,2.0,3.0,175.0,$65.00,$850.00,1,1125,62,38.927572,-77.033604,Washington,20009,DC,187.2


### Error Metrics

We now need a metric that quantifies how good the predictions were on the test set. This class of metrics is called an `error metric.` As the name suggests, an `error metric` quantifies how inaccurate our predictions were from the actual values. In our case, the error metric tells us how off our predicted price values were from the actual price values for the living spaces in the test dataset.

We could start by calculating the difference between each predicted and actual value and then averaging these differences. This is referred to as mean error but isn't an effective error metric for most cases. `Mean error` treats a positive difference differently than a negative difference, but we're really interested in how far off the prediction is in either the positive or negative direction. If the true price was 200 dollars and the model predicted 210 or 190 it's off by 10 dollars either way.

We can instead use the mean absolute error, where we compute the absolute value of each error before we average all the errors.

We can instead use the mean absolute error, where we compute the absolute value of each error before we average all the errors.

### Instructions
1. Use numpy.absolute() to calculate the mean absolute error between predicted_price and price.

2. Assign the MAE to mae

In [14]:
mae = np.abs((test_df['predicted_price'] - test_df['price'])) 
mae.head()
mae = mae.mean()
mae

56.29001074113856

### Mean Squared Error

For many prediction tasks, we want to penalize predicted values that are further away from the actual value far more than those closer to the actual value.

We can instead take the mean of the squared error values, which is called the mean squared error or MSE for short. The MSE makes the gap between the predicted and actual values more clear. A prediction that's off by 100 dollars will have an error (of 10,000) that's 100 times more than a prediction that's off by only 10 dollars (which will have an error of 100).

Here's the formula for MSE:

### Instructions

Calculate the MSE value between the predicted_price and price columns and assign to mse.

In [15]:
mse = np.abs((test_df['price'] - test_df['predicted_price'])**2)
mse = mse.mean() 
mse

18646.525370569278

### Training another model

The model we trained achieved a mean squared error of around `18646.5`. Is this a high or a low mean squared error value? What does this tell us about the quality of the predictions and the model? By itself, the mean squared error value for a single model isn't all that useful.

The units of mean squared error in our case is dollars squared (not dollars), which makes it hard to reason about intuitively as well. We can, however, train another model and then compare the mean squared error values to see which model performs better on a relative basis. Recall that a low error metric means that the gap between the predicted list price and actual list price values is low while a high error metric means the gap is high.

Let's train another model, this time using the bathrooms column, and compare MSE values.


### Instructions
1. Modify the predict_price function to the right to use the bathrooms column instead of the accommodates column to make predictions.
2. Apply the function to test_df and assign the resulting Series object containing the predicted price values to the predicted_price column in test_df.
3. Calculate the squared error between the price and predicted_price columns in test_df and assign the resulting Series object to the squared_error column in test_df.
4. Calculate the mean of the squared_error column in test_df and assign to mse.
5. Use the print function or the variables inspector to display the MSE value

In [31]:
train_df = dc_listings.iloc[0:2792]
test_df = dc_listings.iloc[2792:]
def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bathrooms'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbors_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors_prices.mean()
    return(predicted_price)

In [32]:
test_df['bathroom_predicted_price'] = test_df['bathrooms'].apply(predict_price) 

<ipython-input-32-32459c31a28e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['bathroom_predicted_price'] = test_df['bathrooms'].apply(predict_price)


In [33]:
test_df.head() 

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,bathroom_predicted_price
2792,20%,75%,1,2,Entire home/apt,0.0,1.0,1.0,120.0,NaN,NaN,1,1125,8,38.922187,-77.032475,Washington,20009,DC,138.8
2793,100%,25%,2,3,Entire home/apt,2.0,2.0,1.0,140.0,$75.00,$150.00,2,1125,7,38.931681,-77.044739,Washington,20010,DC,236.6
2794,NaN,NaN,1,4,Entire home/apt,2.0,1.0,1.0,299.0,NaN,NaN,2,1125,5,38.933765,-77.031488,Washington,20010,DC,138.8
2795,100%,100%,1,3,Entire home/apt,1.0,1.0,1.0,85.0,$30.00,$250.00,1,92,2,38.925692,-77.032616,Washington,20009,DC,138.8
2796,100%,100%,1,6,Entire home/apt,2.0,2.0,3.0,175.0,$65.00,$850.00,1,1125,62,38.927572,-77.033604,Washington,20009,DC,236.6


In [34]:
mae = np.abs((test_df['bathroom_predicted_price'] - test_df['price'])) 
mae = mae.mean()
mae

65.64618689581096

In [36]:
mse = np.abs((test_df['price'] - test_df['bathroom_predicted_price'])**2)
mse = mse.mean() 
mse 

18405.44408163265

### Root Mean Squared Error

While comparing MSE values helps us identify which model performs better on a relative basis, it doesn't help us understand if the performance is good enough in general. This is because the units of the MSE metric are squared (in this case, dollars squared). An MSE value of 16377.5 dollars squared doesn't give us an intuitive sense of how far off the model's predictions are from the true price value in dollars.

Root mean squared error is an error metric whose units are the base unit (in our case, dollars). RMSE for short, this error metric is calculated by taking the square root of the MSE value:

### Instructions
Calculate the RMSE value of the model we trained using the bathrooms column and assign it to rmse.

In [37]:
import math
rmse = math.sqrt(mse)
rmse

135.66666532952246

### Comparing MAE and RMSE

The model achieved an RMSE value of approximately 135.6, which implies that we should expect for the model to be off by 135.6 dollars on average for the predicted price values. Given that most of the living spaces are listed at just a few hundred dollars, we need to reduce this error as much as possible to improve the model's usefulness.

We discussed a few different error metrics we can use to understand a model's performance. As we mentioned earlier, these individual error metrics are helpful for comparing models. To better understand a specific model, we can compare multiple error metrics for the same model. This requires a better understanding of the mathematical properties of the error metrics.
If you look at the equation for MAE:

MAE = \frac{1}{n} \sum_{k=1}^{n} \lvert (actual_1 - predicted_1) \rvert + \cdots + \lvert (actual_n - predicted_n) \rvert

you'll notice that that the differences between predicted and actual values grow linearly. A prediction that's off by 10 dollars has a 10 times higher error than a prediction that's off by 1 dollar. If you look at the equation for RMSE, however:



RMSE = \sqrt { \frac{ \sum_{k=1}^{n} (actual_1 - predicted_1)^2 + \cdots + (actual_n - predicted_n)^2 } {n} }

you'll notice that each error is squared before the square root of the sum of all the errors is taken. This means that the individual errors grows quadratically and has a different effect on the final RMSE value.
Let's look at an example using different data entirely. We've created 2 Series objects containing 2 sets of errors and assigned to errors_one and errors_two.

### Instructions
1. Calculate the MAE for errors_one and assign to mae_one.
2. Calculate the RMSE for errors_one and assign to rmse_one.
3. Calculate the MAE for errors_two and assign to mae_two.
4. Calculate the RMSE for errors_two and assign to rmse_two

In [38]:
errors_one = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10])
errors_two = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000])

In [39]:
errors_one 

0      5
1     10
2      5
3     10
4      5
5     10
6      5
7     10
8      5
9     10
10     5
11    10
12     5
13    10
14     5
15    10
16     5
17    10
dtype: int64

In [44]:
mae_one = errors_one.mean() 
mae_one

7.5

In [45]:
mse_one = errors_one**2
mse_one = mse_one.mean()
mse_one

62.5

In [51]:
rmse_one = math.sqrt(mse_one) 
rmse_one

7.905694150420948

In [48]:
mae_two = errors_two.mean() 
mae_two

62.5

In [49]:
mse_two = errors_two**2
mse_two = mse_two.mean()
mse_two

55612.5

In [52]:
rmse_two = math.sqrt(mse_two) 
rmse_two

235.82302686548658

While the MAE (7.5) to RMSE (7.9056941504209481) ratio was about 1:1 for the first list of errors, the MAE (62.5) to RMSE (235.82302686548658) ratio was closer to 1:4 for the second list of errors. In general, we should expect that the MAE value be much less than the RMSE value. The only difference between the 2 sets of errors is the extreme 1000 value in errors_two instead of 10. When we're working with larger data sets, we can't inspect each value to understand if there's one or some outliers or if all of the errors are systematically higher. Looking at the ratio of MAE to RMSE can help us understand if there are large but infrequent errors. You can read more about comparing MAE and RMSE in this wonderful post.

### Syntax
Calculating the (mean squared error) MSE:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)

mse = test_df['squared_error'].mean()
Calculating the (mean absolute error) 

MAE:
test_df['squared_error'] = np.absolute(test_df['predicted_price'] - test_df['price'])

mae = test_df['squared_error'].mean()
Calculating the root mean squared error
(RMSE):
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)

mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)